In [1503]:
import pandas as pd
import math
import io
from datetime import datetime, time
from alive_progress import alive_bar
import os


# Transcript Mapping

### Set File Names and Variables

Set file names here, the upload boxes will appear under each section of the Read Data section. Please make sure that the real filenames match the file names entered here. 

In [ ]:
# Controls if a sample will be used
# Set to [RANDOM, HEAD, TAIL, NONE]
FILTER = "RANDOM"
SIZE = 1000

# Training Records from SumTotal
filename = "2020-2022 Training Records.xlsx"

# Coursename Mapping Setup
USE_COURSE_MAP = True
CMAP_ORIG = "OLD_CNAME"
CMAP_NEW = "NEW_CNAME"
MAP_PATH = "map.csv"

# Saba Courses
COURSE_PATH = "saba_courses.csv"

# Saba People
PEOPLE_PATH = "saba_people.csv"

### Other Global Variables

In [1505]:
USERNAME = "Employee Number"
COURSEID = "OFFERING"
COURSEVERSION = "COURSE_VERSION"
COMPDATE = "Completion Date"
STATUS = "Completion Status"
COURSETITLE = "Activity Name"
PASSFAIL = "Pass/Fail"
ATTSTATUS = "Attendance Status"

FOLDER = "trans_output/"
OUTPUT_PRE = "transcript_out_"
ERRORS_PRE = "transcript_errors_"
MISSING_COURSE = "missing_course_"
MISSING_PEOPLE = "missing_people_"
FILE_EXT = ".csv"
TIME_SUFF = datetime.now().strftime('%Y_%m_%d-%H-%M')

## Read Data

#### Transcript List

In [1504]:
### Note: if using Google Colab, this must be set to the filename of the 
# file you will load. This is a quirk of Colab. 
try:
  import google.colab
  from google.colab import files
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
  print("Colab")
  uploaded = files.upload()
  source = pd.read_excel(io.BytesIO(uploaded.get(filename)))
else:
  print("Not Colab")
  source = pd.read_excel(filename)

Not Colab


#### Mapping Data

In [1510]:
def map_course(mapping_list, orig_df):
    new_df = orig_df
    for _, map_row in mapping_list.iterrows():
        old_course = map_row[CMAP_ORIG]
        new_course = map_row[CMAP_NEW]
        #new_df[[COURSEID]] = new_df[[COURSEID]].replace(old_course, new_course)
        new_df.replace(old_course, new_course, inplace=True)
        #print(old_course, new_course)
    return new_df

In [ ]:
if USE_COURSE_MAP:
  if IN_COLAB:
    print("Colab")
    uploaded = files.upload()
    course_map = pd.read_csv((uploaded.get(MAP_PATH)))
  else:
    print("Not Colab")
    course_map = pd.read_excel(MAP_PATH)

#### Saba Courses

In [ ]:
if IN_COLAB:
  print("Colab")
  uploaded = files.upload()
  courses = pd.read_csv((uploaded.get(COURSE_PATH)))
else:
  print("Not Colab")
  courses = pd.read_excel(COURSE_PATH)


#### Saba People

In [ ]:
if IN_COLAB:
  print("Colab")
  uploaded = files.upload()
  people = pd.read_csv((uploaded.get(PEOPLE_PATH)))
else:
  print("Not Colab")
  people = pd.read_excel(PEOPLE_PATH)

## Begin Processing

In [1506]:
# Generate Output Filename
def genFileName(prefix, suffix, extension):
    cwd = os.getcwd()+"/"
    return cwd+FOLDER+str(prefix)+str(suffix)+str(extension)

In [1507]:
# Take Data Sample if Testing
if FILTER == "RANDOM":
    source = source.sample(SIZE)
elif FILTER == "HEAD":
    source = source.head(SIZE)
elif FILTER == "TAIL":
    source = source.tail(SIZE)


In [1508]:
source.head(5)

,Employee Number,Employee Name,Waived Notes,Employee email,Job Title,Department Code,Department Name,MANAGER,Activity Type,OFFERING,Activity Name,COURSE_VERSION,Score,Attendance Status,Pass/Fail,Completion Status,Completion Date,Expiration Date,Note
152273,91396,"LEACH, Jessica",NaN,jessica.s.leach@cnl.ca,Radiation Surveyor,8660.0,WASTE RP,"HAMILTON, Jay",ILT Class,EMP-1002,EOC - CRL EOC Overview,NaN,NaN,Attended,Pass,Completed,2021-01-13,NaT,NaN
128927,53779,"WOOD, Darryl",NaN,Darryl.Wood@cnl.ca,WMA Senior Operations Technici,454.0,WL SITE & NUCLEAR OPERATIONS,"WALSH, Michael Patrick",ILT Course,MS-1004-WL,Integrated Work Control 5 Core Elements,NaN,NaN,Attended,Pass,Completed,2021-03-08,NaT,NaN
39376,58597,"FELUSHKO, Kent",NaN,Kent.Felushko@cnl.ca,Environmental Technologist,8588.0,WL ENVIRONMENTAL MANAGEMENT,"BETTERIDGE, James",ILT Class,WL-307,Job Safety Analysis,NaN,87.5,Attended,Pass,Completed,2021-02-02,NaT,NaN
143397,60980,"STRUSS, Ashlee",NaN,Ashlee.Zimmer@cnl.ca,Radiation Protection Assistant,380.0,WL RADIATION PROTECTION,"HAYTER, Jeff",ILT Class,WL-308,Integrated Work Control,NaN,92.0,Attended,Pass,Completed,2021-02-10,NaT,NaN
68604,90086,"ROGERS, Cheryl",NaN,cheryl.rogers@cnl.ca,HR Generalist,8463.0,TALENT MANAGEMENT,"ALLEN, Caroline",Versional,FIRE-1001-Online,Fire Prevention,NaN,90.0,Attended,Pass,Completed,2022-01-13,2025-01-13,NaN


### Generate Status Table (Optional)

In [1509]:
#print(source.shape[1])
#source[["Completion Status", "Attendance Status", "Pass/Fail"]].groupby(["Pass/Fail","Completion Status", "Attendance Status"], dropna=False, as_index=False).size()

19


,Pass/Fail,Completion Status,Attendance Status,size
0,Fail,Completed,Attended,38
1,Pass,Completed,Attended,686
2,Pass,Completed,Waived,10
3,Pass,NaN,Attended,1
4,NaN,Completed,Attended,183
5,NaN,NaN,Attended,4
6,NaN,NaN,Cancelled,76
7,NaN,NaN,No-show,2


### Update Mapped Course Names

In [1512]:
if USE_COURSE_MAP:
    source = map_course(course_map, source)
source.head(5)

,Employee Number,Employee Name,Waived Notes,Employee email,Job Title,Department Code,Department Name,MANAGER,Activity Type,OFFERING,Activity Name,COURSE_VERSION,Score,Attendance Status,Pass/Fail,Completion Status,Completion Date,Expiration Date,Note
152273,91396,"LEACH, Jessica",NaN,jessica.s.leach@cnl.ca,Radiation Surveyor,8660.0,WASTE RP,"HAMILTON, Jay",ILT Class,EMP-1002,EOC - CRL EOC Overview,NaN,NaN,Attended,Pass,Completed,2021-01-13,NaT,NaN
128927,53779,"WOOD, Darryl",NaN,Darryl.Wood@cnl.ca,WMA Senior Operations Technici,454.0,WL SITE & NUCLEAR OPERATIONS,"WALSH, Michael Patrick",ILT Course,MS-1004-WL,Integrated Work Control 5 Core Elements,NaN,NaN,Attended,Pass,Completed,2021-03-08,NaT,NaN
39376,58597,"FELUSHKO, Kent",NaN,Kent.Felushko@cnl.ca,Environmental Technologist,8588.0,WL ENVIRONMENTAL MANAGEMENT,"BETTERIDGE, James",ILT Class,WL-307,Job Safety Analysis,NaN,87.5,Attended,Pass,Completed,2021-02-02,NaT,NaN
143397,60980,"STRUSS, Ashlee",NaN,Ashlee.Zimmer@cnl.ca,Radiation Protection Assistant,380.0,WL RADIATION PROTECTION,"HAYTER, Jeff",ILT Class,WL-308,Integrated Work Control,NaN,92.0,Attended,Pass,Completed,2021-02-10,NaT,NaN
68604,90086,"ROGERS, Cheryl",NaN,cheryl.rogers@cnl.ca,HR Generalist,8463.0,TALENT MANAGEMENT,"ALLEN, Caroline",Versional,FIRE-1001-Online,Fire Prevention,NaN,90.0,Attended,Pass,Completed,2022-01-13,2025-01-13,NaN


### Read SABA Data

#### Course Data

In [1513]:
# Process Course Data from Saba
course_id = courses[["Course ID", "Version"]]
ids_only = pd.Series(courses["Course ID"].unique())

courses.head()

,Title,Version,Course ID,Unnamed: 3
0,100035-IBM Cognos Analytics Advanced (Cost Cou...,1.0,IT-9050,NaN
1,100035-IBM Cognos Analytics Fundamentals (Cost...,1.0,IT-9049,NaN
2,10 Commandments of Training,1.0,TD-1048,NaN
3,12 GA Shotgun,1.0,SECU-3015,NaN
4,5S Methodology (Cost Course),1.0,PI-9006-ONLINE,NaN


#### Employee Data

In [1514]:
# Process People Data from Saba
people_un = set(people["Username"].astype("int64", errors="ignore").unique())
#people.head()
len(people_un)

3869

### Data Processing Functions

In [1515]:
def procStatus(pass_fail, comp_status, att_status):
    status = -1
    ## Fail
    if pass_fail == "Fail":
        status = 400
    elif (pass_fail == "Pass") & (comp_status == "Not Completed"):
        status = 400
    #elif (math.isnan(pass_fail)) & (math.isnan(comp_status)) & ((att_status == "No-show") | (att_status == "Replaced")):
    #    status = 400
    ## Pass
    elif (pass_fail == "Pass") & (comp_status != "Not Completed"):
        status = 200
    elif (pd.isnull(pass_fail)) & (comp_status == "Completed"):
        status = 200
        #print("no p/f, status is complete")
    elif ( (pd.isnull(pass_fail)) & (pd.isnull(comp_status)) & (att_status == "Attended") ):
        status = 200
        #print("no p/f, no status, attstatus is attended")
    else:
        status = 400
    return status

def checkPerson(unList, person):
    is_in = False
    if str(person) in unList:
        is_in = True
    return is_in

def checkCourse(courseList, courseID, version):
    is_in = False
    replace_version = version
    for _, row in courseList.iterrows():
        #print(courseID, version, row["Course ID"], row["Version"])
        if (courseID == row["Course ID"]):
            #print("Course Match", courseID, version, row["Course ID"], row["Version"])
            if pd.isnull(version):
                replace_version = row["Version"]
                #print("Insert Version", replace_version, version)
            #else:
                #print("Orig Version:", replace_version, version)
            is_in = True
            #print("Match:", courseID, replace_version, version)
    return is_in, replace_version

class TranscriptRow:
    def __init__(self, username, courseID, courseVersion, compDate, status, courseTitle, inCourse, inPerson):
        self.username = username
        self.courseID = courseID.upper()
        self.courseVersion = courseVersion
        self.compDate = compDate
        self.status = status
        self.courseTitle = courseTitle
        self.inCourse = inCourse
        self.inPerson = inPerson


In [1517]:
# Process Data
source_used = source[[USERNAME, COURSEID, COURSEVERSION, COMPDATE, PASSFAIL, STATUS, ATTSTATUS, COURSETITLE]]

transcripts = []

with alive_bar(source_used.shape[0], force_tty=True) as bar_read:
    for index, trans in source_used.iterrows():
        username = trans[USERNAME]
        courseID = trans[COURSEID].upper()
        course_ver = trans[COURSEVERSION]
        comp_date = trans[COMPDATE].strftime('%Y-%m-%d')
        status = procStatus(trans[PASSFAIL], trans[STATUS], trans[ATTSTATUS])
        course_title = trans[COURSETITLE]
        in_course, rep_ver = checkCourse(course_id, courseID, course_ver)
        in_pers = checkPerson(people_un, username)

        #Update Version
        course_ver = rep_ver
        
        trans_entry = TranscriptRow(username, courseID, course_ver, comp_date, status, course_title, in_course, in_pers)
        transcripts.append(trans_entry)
        #print(courseID, type(courseID), courseID in ids_only.values)
        bar_read()
print("Transcripts Processed:", len(transcripts))

|████████████████████████████████████████| 1000/1000 [100%] in 1:01.7 (16.20/s)                                         a: 1:03) 17/1000 [2%] in 1s (14.7/s, eta: 1:05)  30/1000 [3%] in 2s (15.1/s, eta: 1:04)  ▂▄▆ 52/1000 [5%] in 3s (15.7/s, eta: 1:01) ▆▄▂ 70/1000 [7%] in 4s (16.2/s, eta: 58s)  ▁▃▅ 106/1000 [11%] in 7s (16.2/s, eta: 55s) ▃▅▇ 111/1000 [11%] in 7s (16.2/s, eta: 55s)  174/1000 [17%] in 11s (16.3/s, eta: 51s)  ▃▅▇ 196/1000 [20%] in 12s (16.4/s, eta: 49s)  ▆▄▂ 208/1000 [21%] in 13s (16.4/s, eta: 48s)  212/1000 [21%] in 13s (16.4/s, eta: 48s) 226/1000 [23%] in 14s (16.4/s, eta: 47s)  ▁▃▅ 241/1000 [24%] in 15s (16.4/s, eta: 46s)  ▂▄▆ 264/1000 [26%] in 16s (16.0/s, eta: 46s) (15.8/s, eta: 42s) ▇▅▃ 372/1000 [37%] in 24s (15.6/s, eta: 40s) ▇▇▅ 429/1000 [43%] in 27s (15.8/s, eta: 36s) ▁▃▅ 477/1000 [48%] in 30s (16.0/s, eta: 33s)  ▃▁▃ 504/1000 [50%] in 31s (16.0/s, eta: 31s)  562/1000 [56%] in 35s (16.1/s, eta: 27s) ▃▁▃ 644/1000 [64%] in 40s (16.0/s, eta: 22s)  664/1000 [66%] in 41

### Print to File Outputs

In [1518]:
# Process a Transcript Row
def write_row(df_output, trans, test=False, subset="ALL"):
    #df = df_output
    
    inCourse = trans.inCourse
    inPerson = trans.inPerson

    tmp_dict = {
        "LEARNER": trans.username,
        "COURSE": trans.courseID,
        "COURSE_VERSION": trans.courseVersion,
        "COMPLETION_DATE": trans.compDate,
        "COMPLETION_STATUS": trans.status,
        "CREATE_ADHOC_COURSE": "FALSE",
        "COURSE_TITLE": trans.courseTitle
    }
    if test == True:
        tmp_dict = {
            "LEARNER": trans.username,
            "COURSE": trans.courseID,
            "COURSE_VERSION": trans.courseVersion,
            "COMPLETION_DATE": trans.compDate,
            "COMPLETION_STATUS": trans.status,
            "CREATE_ADHOC_COURSE": "FALSE",
            "COURSE_TITLE": trans.courseTitle,
            "IN_COURSE": trans.inCourse,
            "IN_PERSON": trans.inPerson
        }
    if subset == "ALL":
        df_output = df_output.append(tmp_dict, ignore_index=True)
    elif subset == "CLEAN":
        if ( (inCourse == 1) & (inPerson == 1) ):
            df_output = df_output.append(tmp_dict, ignore_index=True)
    elif subset == "MISSING":
        if ( (inCourse == 0) | (inPerson == 0) ):
            tmp_dict["COURSE_GOOD"] = bool(inCourse)
            tmp_dict["PERSON_GOOD"] = bool(inPerson)
            df_output = df_output.append(tmp_dict, ignore_index=True)
    
    return df_output
    

In [1519]:
# Output Header Values
out_header = [
    "LEARNER",
    "COURSE",
    "COURSE_VERSION",
    "COMPLETION_DATE",
    "COMPLETION_STATUS",
    "CREATE_ADHOC_COURSE",
    "COURSE_TITLE"
]

### Generate Saba Import Data Table and Write to File

In [1520]:
out = pd.DataFrame(columns = out_header)

with alive_bar(len(transcripts), force_tty=True) as bar:
    for ts in transcripts:
        out = write_row(out, ts, False, "CLEAN")
        bar()
out["ID"] = out.index
out.sample(10).T

|████████████████████████████████████████| 1000/1000 [100%] in 1.6s (615.78/s)                                          


,525,304,421,76,290,69,447,494,292,126
LEARNER,92213,2379,60826,61020,53044,53014,61867,56817,57369,55812
COURSE,NMMS-1011-ONLINE,HR-1005,OSH-9045-ONLINE,SECU-3041,SECU-2029,MS-1002,WMA-1002,HR-1005,OSH-9070-ONLINE,OSH-3002-C
COURSE_VERSION,3.6,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
COMPLETION_DATE,2021-06-15,2021-12-08,2021-11-18,2022-04-19,2021-04-15,2021-11-17,2022-01-11,2021-11-05,2021-02-03,2022-01-13
COMPLETION_STATUS,200,200,200,200,200,200,400,200,200,400
CREATE_ADHOC_COURSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
COURSE_TITLE,Nuclear Criticality Safety Program Awareness,Mandatory Harassment and Violence Prevention T...,Hand Safety,Primary Firearms Qualification (Colt C8 Carbine),Security Awareness Refresher,Integrated Work Control - Enhanced,WS BLOCK - Waste Management Program Awareness,Mandatory Harassment and Violence Prevention T...,Arc Flash Safety for Canada,WS BLOCK - Lift Truck Practical - Counter Balance
ID,525,304,421,76,290,69,447,494,292,126


In [1521]:
print(out.shape)
#out.to_csv("out_transcript.csv", index=False)
out.to_csv(genFileName(OUTPUT_PRE, TIME_SUFF, FILE_EXT), index=False)

(558, 8)


#### Write Failiures to File

In [1522]:
out_err = pd.DataFrame(columns = out_header)

with alive_bar(len(transcripts), force_tty=True) as bar:
    for ts in transcripts:
        out_err = write_row(out_err, ts, False, "MISSING")
        bar()
out_err["ID"] = out_err.index
out_err.head(10).T

|████████████████████████████████████████| 1000/1000 [100%] in 1.4s (723.46/s)                                          


,0,1,2,3,4,5,6,7,8,9
LEARNER,56466,10735,54696,60088,50978,57857,58368,59801,91948,54642
COURSE,LGL-1003-ONLINE (REV 3.0),WL-308,OSH-9058-ONLINE (REV 1),OSH-9045-ONLINE (REV 1),FIRE-1001-ONLINE (REV 4.4),OSH-3001-J,CNL-1009-ONLINE (REV 1. 3),EMP-1037-ONLINE (REV 0.1),OSH-9045-ONLINE (REV 1),LGL-1003-ONLINE (REV 3.1)
COURSE_VERSION,3.0,1.0,1.0,1.0,4.4,NaN,1.3,0.1,1.0,3.1
COMPLETION_DATE,2021-10-27,2021-01-19,2021-03-31,2021-08-17,2021-03-19,2021-01-26,2021-08-30,2022-01-18,2021-09-16,2021-11-19
COMPLETION_STATUS,400,200,200,200,200,200,200,200,400,200
CREATE_ADHOC_COURSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
COURSE_TITLE,Values and Ethics at CNL (3.0),Integrated Work Control,Cost Course - Compressed Gas Cylinder Safety (...,Cost Course - Hand Safety (1.0),Fire Prevention (4.4),Crane - Safe Indoor Hoist - Practical - Gantry...,Employee Formal Acknowledgement of CNL Executi...,Emergency Procedure Refresher (0.1),Cost Course - Hand Safety (1.0),Values and Ethics at CNL (3.1)
COURSE_GOOD,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PERSON_GOOD,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
ID,0,1,2,3,4,5,6,7,8,9


In [1523]:
print(out_err.shape)
#out_err.to_csv("out_errors.csv", index=False)
out_err.to_csv(genFileName(ERRORS_PRE, TIME_SUFF, FILE_EXT), index=False)

(442, 10)


### Generate Lists of Missing Courses and People

Generate list of course IDs and usernames that are in the transcript data, but do not exist in Saba. We know these will fail since the requisite data does not exist in Saba. 

In [1524]:
course_need = out_err[out_err["COURSE_GOOD"] == False]
person_need = out_err[out_err["PERSON_GOOD"] == False]

#pd.Series(course_need["COURSE"].unique(), name="Missing Courses").to_csv("out_courses.csv")
#pd.Series(person_need["LEARNER"].unique(), name="Missing Learners").to_csv("out_people.csv")
pd.Series(course_need["COURSE"].unique(), name="Missing Courses").to_csv(genFileName(MISSING_COURSE, TIME_SUFF, FILE_EXT))
pd.Series(person_need["LEARNER"].unique(), name="Missing Learners").to_csv(genFileName(MISSING_PEOPLE, TIME_SUFF, FILE_EXT))